In [1]:
# Core Modules
import pandas as pd
import numpy as np

# Basic modules for data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [2]:
test_df = pd.read_csv('clean_test.csv')
train_df = pd.read_csv('clean_train.csv')

In [4]:
train_df.shape

(1421, 71)

In [3]:
test_df.shape

(1459, 70)

In [6]:
final_df = pd.concat([train_df,test_df],axis=0)
final_df.shape

(2880, 71)

In [7]:
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields],drop_first=True)
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final,df1],axis=1)
        i = i + 1

    df_final = pd.concat([final_df,df_final],axis = 1)
    return df_final

In [9]:
cat_attributes = final_df.select_dtypes(include='object')

In [10]:
final_df_1hot = final_df

final_df_1hot = category_onehot_multcols(cat_attributes)

MSZoning
Street
Alley
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [11]:
final_df_1hot.shape

(2880, 243)

In [12]:
final_df_1hot = final_df_1hot.loc[:,~final_df_1hot.columns.duplicated()]
final_df_1hot.shape

(2880, 167)

In [15]:
df_train = final_df_1hot.iloc[:1421,:]
df_test = final_df_1hot.iloc[1421:,:]

In [16]:
df_train.shape

(1421, 167)

In [17]:
df_test.shape

(1459, 167)

In [19]:
df_test.drop(['SalePrice'],axis=1,inplace=True)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
x_train = df_train.drop(['SalePrice'],axis=1)
y_train = df_train['SalePrice']

In [21]:
from sklearn.ensemble import RandomForestRegressor

rf_model_final = RandomForestRegressor(bootstrap=False,max_features=40, n_estimators=100, random_state=5)

rf_model_final.fit(x_train, y_train)

RandomForestRegressor(bootstrap=False, max_features=40, random_state=5)

In [22]:
rf_model_final_pred = rf_model_final.predict(df_test)

In [23]:
pred = pd.DataFrame(rf_model_final_pred)
sub_df = pd.read_csv('../../Datasets/house-prices-advanced-regression-techniques/sample_submission.csv')
datasets = pd.concat([sub_df['Id'],pred],axis=1)

In [24]:
datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      1459 non-null   int64  
 1   0       1459 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


In [25]:
datasets.columns = ['Id', 'SalePrice']
datasets.to_csv('submission.csv',index=False)